In [ ]:
import numpy as np
import cv2
from imutils import face_utils
import imutils
import dlib

import tensorflow.keras
from tensorflow.keras import models, optimizers
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# Facial emotion recognition

## Objectives

* Use a NN fully connected based on euclidian distance
* Use a CNN based on images

In [ ]:
# Path where are the following files :
# x_train, y_train, images_train, labels_train
# haarcascade_frontalface_default.xml
# shape_predictor_68_face_landmarks.dat

DATA_PATH = "../bmdata/data/TP_TP4"

# Prepare class for building the different models

## Attributes
* path_to_data : where data will be loaded. Here, typicaly DATA_PATH
* model_type : "CNN" or "Dense"
* model : the model we'll create and use
* X_train, y_train : data to fit the model

## Methods
* init : initialize the model depending of its type. Load train data.
* compile_model : compile the model and show the summary
* fit : fit the model and return history
* load_weights : load the weights saved on a previous model based on model_type
* predict : return the prediction for the given parameter

In [ ]:
class EmotionModel():
    def __init__(self, path_to_data, model_type = "Dense"):
        model = models.Sequential()
        self.path_to_data = path_to_data

        if model_type == "Dense":
            self.model_type = "Dense"
            model.add(Dense(512, activation='relu', input_shape=(4624,)))
            model.add(Dense(512, activation='relu'))
            model.add(Dense(512, activation='relu'))
            model.add(Dense(512, activation='relu'))
            model.add(Dense(512, activation='relu'))
            model.add(Dense(7, activation='softmax'))
            
            # Preload data to fit
            self.X_train = np.load(self.path_to_data + "/x_train.npy")
            self.y_train = np.load(self.path_to_data + "/y_train.npy")

        elif model_type == "CNN":
            self.model_type = "CNN"
            model.add(Conv2D(16, 3, activation='relu', input_shape=(48,48,1)))
            model.add(Conv2D(32, 3, activation='relu'))
            model.add(Conv2D(64, 3, activation='relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(128, 3, activation='relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Flatten())
            model.add(Dropout(0.2))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(7, activation='softmax'))
            
            # Preload data to fit
            self.X_train = np.load(self.path_to_data + "/images_train.npy")
            print(self.x_train)
            self.y_train = np.load(self.path_to_data + "/labels_train.npy")
        
        self.model = model
        
    def compile_model(self, loss, optimizer, metrics):
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        self.model.summary()
    
    def fit(self, validation_split=0.2, epochs=200, batch_size=128):
        # Shuffle training data
        X_train, y_train = shuffle(self.X_train, self.y_train)
        
        # Fit the model and get the hostory
        history = self.model.fit(X_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size)
        
        # Save the weights in a .h5 file
        self.model.save_weights(self.path_to_data + "/weights_" + self.model_type + ".h5")
        return history
    
    def load_weights(self):
        self.model.load_weights(self.path_to_data + "/weights_" + self.model_type + ".h5")
    
    def predict(self, distances):
        prediction = self.model.predict(distances)
        return prediction

## Choose for model type you want to use

In [ ]:
# Choose between "CNN" or "Dense"
model_type = "CNN"

## Create model, compile and show summary

In [ ]:
model = EmotionModel(DATA_PATH, model_type)

params = {
            "CNN_loss": "mean_squared_error", "CNN_optimizer": "adam",
            "Dense_loss": "categorical_crossentropy", "Dense_optimizer": optimizers.RMSprop(lr=1e-5)
         }

model.compile_model(loss=params[model_type+"_loss"], optimizer=params[model_type+"_optimizer"], metrics=['acc'])

## Fit

In [ ]:
# Fit the model
history = model.fit(validation_split=0.2, epochs=20, batch_size=32)

# Plot curves for accuracy and loss after fitting
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Directly load the weights for the model based on previous learning
model.load_weights()

# Functions used

## Functions
* get_emotion : return emotion in string format from the vector
* detect_parts : calculate distances between points and return an array of distances

In [ ]:
def get_emotion(emotion):
    emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise')
    emo = emotions[np.argmax(emotion)]
    return emo

def detect_parts(image):
    # resize the image, and convert it to grayscale
    image = imutils.resize(image, width=200, height=200)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale image
    rects = detector(gray, 1)
    
    distances = np.zeros((68,68))
    
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        distances = np.linalg.norm(shape - shape[:,None], axis=-1)
        distances = (distances - np.mean(distances))/ 3*np.std(distances)
    return distances

# Live emotion recognizer

* Initialize the camera and classifiers
* Detect faces
* Depending on the model, use images or distances to predict the emotion

In [ ]:
# -----------------------------
# opencv initialization
face_cascade = cv2.CascadeClassifier(DATA_PATH + "/haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

# initialize dlib's face detector and create a predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(DATA_PATH + "/shape_predictor_68_face_landmarks.dat")
    
while (True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  
        # draw rectangle to main image
        detected_face = img[int(y):int(y + h), int(x):int(x + w)]  
        # crop detected face
        
        if model_type == "Dense":
            distances = detect_parts(detected_face)
            emotion = model.predict(distances.reshape((-1, 4624)))
            emotion_text = get_emotion(emotion)
        else:
            image = gray[int(y):int(y + h), int(x):int(x + w)]
            image = np.resize(image, (48,48))
            emotion = model.predict((np.reshape(image, (1, 48, 48, 1)))/np.max(image))
            emotion_text = get_emotion(emotion)
        # write emotion text above rectangle
        cv2.putText(img, emotion_text, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('img', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  
        # press q to quit
        break
        # kill opencv
cap.release()
cv2.destroyAllWindows()

# Result analysis

The two models aren't working very well. After tests, the prediction depends on :
* the shape of the face
* the light (lamp, sun, clouds)
* the normalization (concerning CNN)